In [35]:
using SparseArrays
using LinearAlgebraicRepresentation

Lar=LinearAlgebraicRepresentation
function spatial_arrangement_1(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model)

	# initializations
    fs_num = size(copFE, 1)
    rV = Array{Float64,2}(undef,0,3)
    rEV = SparseArrays.spzeros(Int8,0,0)
    rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(100))
        @async begin
            for sigma in 1:fs_num
                
                put!(in_chan, sigma)
                @show sigma
            end
            for p in Distributed.workers()
               @show p
               put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            
            @async Base.remote_do(frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
            
        end
        
        for sigma in 1:fs_num
                rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
           
    
        end
    else
	# sequential (iterative) processing of face fragmentation
        for sigma in 1:fs_num
            #print(sigma, "/", fs_num, "\r")
            nV, nEV, nFE = Lar.Arrangement.frag_face(V, copEV, copFE, sp_idx, sigma)
			#nV, nEV, nFE = Lar.fragface(V, copEV, copFE, sp_idx, sigma)
			nV = convert(Lar.Points, nV)
            a,b,c = Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            rV=a;  rEV=b;  rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = Lar.Arrangement.merge_vertices(rV, rEV, rFE)
    return rV, rEV, rFE
end

spatial_arrangement_1 (generic function with 2 methods)

In [6]:
using SparseArrays
using LinearAlgebraicRepresentation

Lar=LinearAlgebraicRepresentation
function spatial_arrangement_1(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model)

	# initializations
    fs_num = size(copFE, 1)
    rV = Array{Float64,2}(undef,0,3)
    rEV = SparseArrays.spzeros(Int8,0,0)
    rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
         rV,rEV,rFE=spatial_arrangement_1distributed(V,copEV,copFE)
        
       
        
    else
	# sequential (iterative) processing of face fragmentation
        rV,rEV,rFE=spatial_arrangement_1sequential(V,copEV,copFE)
    end


	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = Lar.Arrangement.merge_vertices(rV, rEV, rFE)
    return rV, rEV, rFE
end

spatial_arrangement_1 (generic function with 2 methods)

In [7]:
function spatial_arrangement_1sequential(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp)
    
    # spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model)

	# initializations
    fs_num = size(copFE, 1)
    rV = Array{Float64,2}(undef,0,3)
    rEV = SparseArrays.spzeros(Int8,0,0)
    rFE = SparseArrays.spzeros(Int8,0,0)
    
     for sigma in 1:fs_num
            #print(sigma, "/", fs_num, "\r")
            nV, nEV, nFE = Lar.Arrangement.frag_face(V, copEV, copFE, sp_idx, sigma)
			#nV, nEV, nFE = Lar.fragface(V, copEV, copFE, sp_idx, sigma)
			nV = convert(Lar.Points, nV)
            a,b,c = Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            rV=a;  rEV=b;  rFE=c
        end
    
    return rV,rEV,rFE
end

spatial_arrangement_1sequential (generic function with 1 method)

In [19]:
using Distributed
addprocs(2)
@everywhere using LinearAlgebraicRepresentation
@everywhere Lar=LinearAlgebraicRepresentation

@everywhere function spatial_arrangement_1distributed(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model)

	# initializations
    fs_num = size(copFE, 1)
    rV = Array{Float64,2}(undef,0,3)
    
    rEV = SparseArrays.spzeros(Int8,0,0)
    
    rFE = SparseArrays.spzeros(Int8,0,0)
    
    
            in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
            out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(100))
       
            #@async begin
            @async begin
            for sigma in 1:fs_num
                
                put!(in_chan, sigma)
                
                
            end
            for p in Distributed.workers()
               @show Distributed.workers()
               put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            
            @async remote_do(Lar.Arrangement.frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
            @show p
            
           
        end
        
        for sigma in 1:fs_num
         a,b,c = Lar.skel_merge( rV,rEV,rFE, take!(out_chan)... )
          rV=a;  rEV=b;  rFE=c
        end
     
     
     
     return rV, rEV, rFE
    
    
    
    

end

LoadError: InterruptException:

In [20]:

using DistributedArrays


using Distributed
using SparseArrays
using LinearAlgebraicRepresentation

Lar=LinearAlgebraicRepresentation
V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

    EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
    ])

    FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
    ])

@time spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true)


p = 2
p = 3
p = 4
p = 5
p = 6
p = 7
Distributed.workers() = [2, 3, 4, 5, 6, 7]
  0.036700 seconds (7.34 k allocations: 426.289 KiBDistributed.workers() = )[2, 3, 4, 5, 6, 7]



([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [9 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [10, 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [11, 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [12, 4]  =  -1
  [5 , 5]  =  -1
  [8 , 5]  =  -1
  [9 , 5]  =  1
  [5 , 6]  =  1
  [6 , 6]  =  -1
  [10, 6]  =  1
  [6 , 7]  =  1
  [7 , 7]  =  -1
  [11, 7]  =  1
  [7 , 8]  =  1
  [8 , 8]  =  1
  [12, 8]  =  1, 
  [1,  1]  =  1
  [3,  1]  =  1
  [1,  2]  =  1
  [4,  2]  =  1
  [1,  3]  =  1
  [5,  3]  =  1
  [1,  4]  =  -1
  [6,  4]  =  1
  [2,  5]  =  1
  [3,  5]  =  -1
  [2,  6]  =  1
  [4,  6]  =  -1
  [2,  7]  =  1
  [5,  7]  =  -1
  [2,  8]  =  -1
  [6,  8]  =  -1
  [3,  9]  =  -1
  [6,  9]  =  -1
  [3, 10]  =  1
  [4, 10]  =  -1
  [4, 11]  =  1
  [5, 11]  =  -1
  [5, 12]  =  1
  [6, 12]  =  1)

In [ ]:
using BenchmarkTools
using Pkg
Pkg.instantiate()
@btime spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true)

p = 2
p = 3
p = 4
p = 5
p = 6
p = 7
Distributed.workers() = [2, 3, 4, 5, 6, 7]
p = 2
p = 3
p = 4
p = 5
p = 6
p = 7
Distributed.workers() = [2, 3, 4, 5, 6, 7]
Distributed.workers() = [2, 3, 4, 5, 6, 7]
p = 2
p = 3
p = 4
p = 5
p = 6
p = 7
Distributed.workers() = [2, 3, 4, 5, 6, 7]
p = 2
p = 3
p = 4
p = 5
p = 6
p = 7
Distributed.workers() = [2, 3, 4, 5, 6, 7]


In [ ]:
@btime spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)

In [12]:
@btime spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)

  7.467 ms (56474 allocations: 3.23 MiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [9 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [10, 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [11, 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [12, 4]  =  -1
  [5 , 5]  =  -1
  [8 , 5]  =  -1
  [9 , 5]  =  1
  [5 , 6]  =  1
  [6 , 6]  =  -1
  [10, 6]  =  1
  [6 , 7]  =  1
  [7 , 7]  =  -1
  [11, 7]  =  1
  [7 , 8]  =  1
  [8 , 8]  =  1
  [12, 8]  =  1, 
  [1,  1]  =  1
  [3,  1]  =  1
  [1,  2]  =  1
  [4,  2]  =  1
  [1,  3]  =  1
  [5,  3]  =  1
  [1,  4]  =  -1
  [6,  4]  =  1
  [2,  5]  =  1
  [3,  5]  =  -1
  [2,  6]  =  1
  [4,  6]  =  -1
  [2,  7]  =  1
  [5,  7]  =  -1
  [2,  8]  =  -1
  [6,  8]  =  -1
  [3,  9]  =  -1
  [6,  9]  =  -1
  [3, 10]  =  1
  [4, 10]  =  -1
  [4, 11]  =  1
  [5, 11]  =  -1
  [5, 12]  =  1
  [6, 12]  =  1)